In [9]:
# Script 1
# Import required Libraries
import hvplot.pandas
import matplotlib.pyplot as plt
import pandas as pd
import requests
from scipy.stats import linregress
import json

# Import API keys
from config import geoapify_key
from config import weather_api_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [10]:
# Script 2
# Load HME projects info into a Dataframe
projects_df = pd.read_csv("../project-1/HMEOpenLargeJobs_20231202.csv")

# Display sample data
projects_df.head()

,ProjectID,Project-Number,Name,Address,Geometry,LocationJSON
0,85216,18-054,PAR 1244,"5950 York St, Denver, CO 80216, USA","{""location"":{""lat"":39.805279,""lng"":-104.958759...","{""Street1"":""5950 York St"",""Street2"":null,""City..."
1,87261,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,"6510 Southmoor Dr, Fountain, CO 80817, USA","{""location"":{""lat"":38.736055,""lng"":-104.737949...","{""Street1"":""6510 Southmoor Dr"",""Street2"":null,..."
2,87538,18-145,WESTERN CURRENCY FACILITY EXPANSION,"8776 Blue Mound Rd, Fort Worth, TX 76131, USA","{""location"":{""lat"":32.8943727,""lng"":-97.348633...","{""Street1"":""8776 Blue Mound Rd"",""Street2"":null..."
3,87544,18-147,BID PACKAGE 4.3 - LEONARD WTP YARD PIPE,"361 Co Rd 4965, Leonard, TX 75452, USA","{""location"":{""lat"":33.383391,""lng"":-96.2825639...","{""Street1"":""361 Co Rd 4965"",""Street2"":null,""Ci..."
4,88609,19-040,SADDLE CREEK RTB,"2520 S 64th St, Omaha, NE 68106, USA","{""location"":{""lat"":41.2404302,""lng"":-96.011843...","{""Street1"":""2520 S 64th St"",""Street2"":null,""Ci..."


In [3]:
# current csv HMEOpenLargeJobs_20231202 column headers ProjectID, Project-Number, Name, Address, Geometry, LocationJSON

# create projects_clean_df with project-numbr, Name, address, lat, lng

# show projects_clean_df
# projects_clean_df

In [11]:
# Script 3

import pandas as pd
import json

# Load the data into a DataFrame (assuming the CSV file is in the same directory as the script)
projects_df = pd.read_csv("HMEOpenLargeJobs_20231202.csv")

# Define a function to parse the 'Geometry' column for latitude and longitude
def extract_lat_lng(geometry_str):
    try:
        geometry = json.loads(geometry_str)
        return geometry['location']['lat'], geometry['location']['lng']
    except (json.JSONDecodeError, KeyError):
        return None, None

# Apply the function to extract 'Lat' and 'Lng'
projects_df[['Lat', 'Lng']] = projects_df['Geometry'].apply(lambda x: pd.Series(extract_lat_lng(x)))

# Select the required columns and add empty columns
projects_clean_df = projects_df[['Project-Number', 'Name', 'Address', 'Lat', 'Lng']].copy()
projects_clean_df['Weather_Category'] = ''
projects_clean_df['Color'] = ''
projects_clean_df['Temperature_F'] = ''
projects_clean_df['Windspeed_mph'] = ''

# Remove rows with invalid or missing Lat and Lng
projects_clean_df.dropna(subset=['Lat', 'Lng'], inplace=True)

# Display the cleaned DataFrame
projects_clean_df.head()


,Project-Number,Name,Address,Lat,Lng,Weather_Category,Color,Temperature_F,Windspeed_mph
0,18-054,PAR 1244,"5950 York St, Denver, CO 80216, USA",39.805279,-104.958759,,,,
1,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,"6510 Southmoor Dr, Fountain, CO 80817, USA",38.736055,-104.737949,,,,
2,18-145,WESTERN CURRENCY FACILITY EXPANSION,"8776 Blue Mound Rd, Fort Worth, TX 76131, USA",32.894373,-97.348633,,,,
3,18-147,BID PACKAGE 4.3 - LEONARD WTP YARD PIPE,"361 Co Rd 4965, Leonard, TX 75452, USA",33.383391,-96.282564,,,,
4,19-040,SADDLE CREEK RTB,"2520 S 64th St, Omaha, NE 68106, USA",41.240430,-96.011843,,,,


In [13]:
# Script 4

# Set weather API base URL
weather_api_base_url = "http://api.openweathermap.org/data/2.5/weather?"


In [14]:
# Script 5

import requests
from config import weather_api_key  # Assuming weather_api_key is defined in Script 1

def fetch_weather_data(lat, lng):
    # Building the weather API request URL
    url = f"{weather_api_base_url}lat={lat}&lon={lng}&appid={weather_api_key}&units=imperial"
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            weather_conditions = [weather['main'].lower() for weather in data.get('weather', [])]
            wind_speed = round(data.get('wind', {}).get('speed', 0))
            temp = round(data.get('main', {}).get('temp', 0))

            # Categorize weather conditions
            if 'rain' in weather_conditions and wind_speed > 15:
                return temp, wind_speed, 'Wind and Rain', 'Red'
            elif 'snow' in weather_conditions and wind_speed > 15:
                return temp, wind_speed, 'Wind and Snow', 'Red'
            elif wind_speed > 15:
                return temp, wind_speed, 'High Wind', 'Yellow'
            elif 'rain' in weather_conditions:
                return temp, wind_speed, 'Rain', 'Red'
            elif 'snow' in weather_conditions:
                return temp, wind_speed, 'Snow', 'Red'
            elif temp < 32:
                return temp, wind_speed, 'Cold but Good Weather', 'Blue'
            else:
                return temp, wind_speed, 'Good Weather', 'Green'
        else:
            return None, None, 'Unknown', 'Gray'
    except requests.RequestException:
        return None, None, 'Error', 'Gray'

# Update the DataFrame with weather data
for index, row in projects_clean_df.iterrows():
    temp, wind_speed, weather_category, color = fetch_weather_data(row['Lat'], row['Lng'])
    projects_clean_df.at[index, 'Temperature_F'] = temp
    projects_clean_df.at[index, 'Windspeed_mph'] = wind_speed
    projects_clean_df.at[index, 'Weather_Category'] = weather_category
    projects_clean_df.at[index, 'Color'] = color

# Display the updated DataFrame
pd.set_option('display.max_rows', None)
projects_clean_df



,Project-Number,Name,Address,Lat,Lng,Weather_Category,Color,Temperature_F,Windspeed_mph
0,18-054,PAR 1244,"5950 York St, Denver, CO 80216, USA",39.805279,-104.958759,Good Weather,Green,42,6
1,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,"6510 Southmoor Dr, Fountain, CO 80817, USA",38.736055,-104.737949,Good Weather,Green,44,10
2,18-145,WESTERN CURRENCY FACILITY EXPANSION,"8776 Blue Mound Rd, Fort Worth, TX 76131, USA",32.894373,-97.348633,Good Weather,Green,49,6
3,18-147,BID PACKAGE 4.3 - LEONARD WTP YARD PIPE,"361 Co Rd 4965, Leonard, TX 75452, USA",33.383391,-96.282564,Good Weather,Green,47,3
4,19-040,SADDLE CREEK RTB,"2520 S 64th St, Omaha, NE 68106, USA",41.240430,-96.011843,Good Weather,Green,36,6
5,19-048,MARCY GULCH WWTP,"8700 US-85, Highlands Ranch, CO 80126, USA",39.556900,-105.033535,Good Weather,Green,41,6
6,19-128,UNITED IAH TERMINAL C BHS REPLACEMENT,"3575 S Terminal Rd, Houston, TX 77032, USA",29.985962,-95.336822,Good Weather,Green,54,9
7,19-173,NORTHWATER RAPID MIX/FLOC/SED,"6949 CO-93, Golden, CO 80403, USA",39.818091,-105.221739,Good Weather,Green,40,6
8,20-026,Project MP-18002/19003 Elevated Tanks Water One,"19770 S Lackman Rd, Spring Hill, KS 66083, USA",38.769805,-94.761545,Good Weather,Green,33,5
9,20-133,WWTP Improvements - Tahlequah,"1410 E Powell Rd, Tahlequah, OK 74464, USA",35.892527,-94.954679,Good Weather,Green,50,0


In [15]:

# Save the dataframe to a CSV file in the current folder
csv_file_name = "projects_weather_data.csv"
projects_clean_df.to_csv(csv_file_name, index=False)


In [16]:
!pip install folium


In [17]:
# Script 6

import folium

# Create a map centered on the approximate geographic center of the contiguous US
map_us = folium.Map(location=[39.8283, -98.5795], zoom_start=5)

# Add markers to the map for each project location
for index, row in projects_clean_df.iterrows():
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=5,
        color=row['Color'] if row['Color'] else 'black',  # Default to black if no color specified
        fill=True,
        fill_color=row['Color'] if row['Color'] else 'black',
        fill_opacity=0.7
    ).add_to(map_us)

# Display the map
map_us



In [ ]:
# Configure the map plot
   ## if rain and/or snow color red

# Display map

In [ ]:
# Configure the map plot
   ## wind greater than 25 color yellow

# Display map

In [ ]:
# Configure the map plot
    # if rain and/or snow color red, if high wind color yellow, if rain/snow/wind color red,
    # else if good weather color green

# Display map

In [18]:
# Script 7

import folium
from datetime import datetime
import branca

# Create a map centered around the average coordinates
average_lat = projects_clean_df['Lat'].mean()
average_lng = projects_clean_df['Lng'].mean()
map_us = folium.Map(location=[average_lat, average_lng], zoom_start=4)

# Add markers to the map with hover information
for _, row in projects_clean_df.iterrows():
    hover_text = (f"Project-Number: {row['Project-Number']}<br>"
                  f"Name: {row['Name']}<br>"
                  f"Address: {row['Address']}<br>"
                  f"Weather Category: {row['Weather_Category']}<br>"
                  f"Temperature: {row['Temperature_F']}°F<br>"
                  f"Wind Speed: {row['Windspeed_mph']} mph")
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=5,
        color=row['Color'] if row['Color'] else 'black',  # Default to black if no color specified
        fill=True,
        fill_color=row['Color'] if row['Color'] else 'black',
        fill_opacity=0.7,
        tooltip=hover_text
    ).add_to(map_us)

# Adding a title to the map using an HTML element
title_html = '''
             <h3 align="center" style="font-size:20px"><b>HME, Inc. Project Locations</b></h3>
             <h4 align="center" style="font-size:16px">{}</h4>
             '''.format(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
map_us.get_root().html.add_child(folium.Element(title_html))

# Display the map
map_us


In [69]:
# Script 8

# ... (previous code remains the same)

# List to store weather data
weather_data_list = []

# Populate the list with weather data
for _, project in projects_clean_df.iterrows():
    forecast_data = fetch_5day_3hour_forecast(project['Lat'], project['Lng'])
    if forecast_data:
        construction_weather_days = check_construction_weather_day(forecast_data)
        for weather_date, weather_type in construction_weather_days:
            weather_data_list.append({
                'Project-Number': project['Project-Number'],
                'Name': project['Name'],
                'Lat': project['Lat'],
                'Lng': project['Lng'],
                'Weather_Date': weather_date,
                'Weather_Type': weather_type
            })
    else:
        print(f"No data for project {project['Project-Number']}")

# Create DataFrame from the list
Weather_5Day_Forecast = pd.DataFrame(weather_data_list)

# Display the DataFrame
Weather_5Day_Forecast


,Project-Number,Name,Lat,Lng,Weather_Date,Weather_Type
0,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,38.736055,-104.737949,2024-01-04 12:00:00,Snow
1,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,38.736055,-104.737949,2024-01-04 15:00:00,Snow
2,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,38.736055,-104.737949,2024-01-04 18:00:00,Snow
3,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,38.736055,-104.737949,2024-01-06 09:00:00,Snow
4,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,38.736055,-104.737949,2024-01-06 12:00:00,Snow
5,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,38.736055,-104.737949,2024-01-06 15:00:00,Snow
6,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,38.736055,-104.737949,2024-01-06 18:00:00,Snow
7,18-145,WESTERN CURRENCY FACILITY EXPANSION,32.894373,-97.348633,2024-01-02 18:00:00,Rain
8,18-145,WESTERN CURRENCY FACILITY EXPANSION,32.894373,-97.348633,2024-01-04 15:00:00,High Wind
9,18-145,WESTERN CURRENCY FACILITY EXPANSION,32.894373,-97.348633,2024-01-05 06:00:00,Rain


In [20]:
# Script 8

import requests
import pandas as pd
from datetime import datetime
from config import weather_api_key  # Ensure this is defined in Script 1

# Function to fetch 5-day/3-hour forecast data
def fetch_5day_3hour_forecast(lat, lng):
    url = f"http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lng}&appid={weather_api_key}&units=imperial"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to fetch data: HTTP {response.status_code}")
            return None
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return None

# Function to check for construction weather days
def check_construction_weather_day(forecast_data):
    construction_weather_days = []
    for entry in forecast_data.get('list', []):
        timestamp = entry['dt']
        date = datetime.fromtimestamp(timestamp)
        temp = entry['main']['temp']
        wind_speed = entry['wind']['speed']
        weather_conditions = [weather['main'].lower() for weather in entry['weather']]
        
        if 'rain' in weather_conditions and (6 <= date.hour <= 18):
            construction_weather_days.append((date.strftime("%Y-%m-%d %H:%M:%S"), 'Rain'))
        if 'snow' in weather_conditions and (6 <= date.hour <= 18):
            construction_weather_days.append((date.strftime("%Y-%m-%d %H:%M:%S"), 'Snow'))
        if wind_speed > 15 and (6 <= date.hour <= 18):
            construction_weather_days.append((date.strftime("%Y-%m-%d %H:%M:%S"), 'High Wind'))
        if temp <= 20 and (6 <= date.hour <= 18):
            construction_weather_days.append((date.strftime("%Y-%m-%d %H:%M:%S"), 'Extreme Low Temperature'))
    
    return construction_weather_days

# List to store weather data
weather_data_list = []

# Populate the list with weather data
for _, project in projects_clean_df.iterrows():
    forecast_data = fetch_5day_3hour_forecast(project['Lat'], project['Lng'])
    if forecast_data:
        construction_weather_days = check_construction_weather_day(forecast_data)
        for weather_date, weather_type in construction_weather_days:
            weather_data_list.append({
                'Project-Number': project['Project-Number'],
                'Name': project['Name'],
                'Lat': project['Lat'],
                'Lng': project['Lng'],
                'Weather_Date': weather_date,
                'Weather_Type': weather_type
            })
    else:
        print(f"No data for project {project['Project-Number']}")

# Create DataFrame from the list
Weather_5Day_Forecast = pd.DataFrame(weather_data_list)

# Display the DataFrame
Weather_5Day_Forecast


,Project-Number,Name,Lat,Lng,Weather_Date,Weather_Type
0,18-054,PAR 1244,39.805279,-104.958759,2024-01-08 06:00:00,Snow
1,18-054,PAR 1244,39.805279,-104.958759,2024-01-08 09:00:00,Snow
2,18-054,PAR 1244,39.805279,-104.958759,2024-01-08 09:00:00,High Wind
3,18-054,PAR 1244,39.805279,-104.958759,2024-01-08 12:00:00,Snow
4,18-054,PAR 1244,39.805279,-104.958759,2024-01-08 12:00:00,High Wind
5,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,38.736055,-104.737949,2024-01-04 18:00:00,Snow
6,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,38.736055,-104.737949,2024-01-05 18:00:00,Snow
7,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,38.736055,-104.737949,2024-01-06 06:00:00,Snow
8,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,38.736055,-104.737949,2024-01-06 09:00:00,Snow
9,18-121,SECURITY SANITATION WWTF IMPROVEMENTS,38.736055,-104.737949,2024-01-06 12:00:00,Snow


In [21]:

# Save the dataframe to a CSV file in the current folder
csv_file_name_8a = "Weather_5Day_Forecast.csv"
Weather_5Day_Forecast.to_csv(csv_file_name_8a, index=False)


In [ ]:
# List the total number of projects with rain/snow and/or wind over 25mph

In [ ]:
# Create bar chart with percentage of projects with rain/snow and/or winds over 25mph

In [ ]:
# Create scatter plot rain/snow and/or winds over 25mph Vs Latitude

In [ ]:
# Create scatter plot rain/snow and/or winds over 25mph Vs Longitude

In [ ]:
# List the total number of projects with predicted rain and/or winds over 25mph in the next 8 days

In [ ]:
# Create pie chart for predicted weather
    # if rain and/or snow color red, if high wind color yellow, if rain/snow/wind color red,
    #else if good weather color green